# **Proyek Predictive Analytics: Prediksi Penyakit Jantung**

- **Nama:** Muhammad Husain Fadhlillah
- **Email Student:** mc006d5y2343@student.devacademy.id
- **Cohort ID:** MC006D5Y2343

## **1. Mengimpor Library yang Dibutuhkan**
Langkah pertama adalah mengimpor semua library Python yang akan digunakan.

In [ ]:
# Library untuk data manipulation dan analysis
import pandas as pd
import numpy as np

# Library untuk visualisasi data
import matplotlib.pyplot as plt
import seaborn as sns

# Library dari Scikit-learn untuk machine learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Mengatur agar plot ditampilkan inline di notebook
%matplotlib inline

## **2. Data Loading dan Understanding**
Pada tahap ini akan memuat dataset, memberinya nama kolom yang sesuai, dan melakukan eksplorasi data awal untuk memahami strukturnya.

In [ ]:
# URL dari dataset Heart Disease UCI
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data"

# Nama kolom sesuai dengan dokumentasi dataset
column_names = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
                'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target']

# Memuat dataset menggunakan pandas
# Dataset ini tidak memiliki header, dan missing values ditandai dengan '?'
df = pd.read_csv(url, header=None, names=column_names, na_values='?')

# Menampilkan 5 baris pertama dari dataframe
print("5 baris pertama data:")
display(df.head())

# Mengubah nilai target: nilai > 0 berarti ada penyakit jantung (diubah menjadi 1)
df['target'] = (df['target'] > 0).astype(int)

## **3. Exploratory Data Analysis (EDA)**
Melakukan EDA untuk mendapatkan wawasan lebih dalam dari data, seperti memeriksa nilai yang hilang, melihat statistik deskriptif, dan memvisualisasikan distribusi serta korelasi data.

In [ ]:
# Menampilkan informasi dasar dataframe
print("\nInformasi DataFrame:")
df.info()

# Memeriksa jumlah nilai yang hilang di setiap kolom
print("\nJumlah Missing Values per Kolom:")
print(df.isnull().sum())

# Karena jumlah missing values sedikit (4 pada 'ca' dan 2 pada 'thal'),
# kita akan menghapus baris yang mengandung missing values.
df.dropna(inplace=True)

# Memeriksa ulang missing values
print("\nJumlah Missing Values setelah dihapus:")
print(df.isnull().sum())

# Menampilkan statistik deskriptif untuk fitur numerik
print("\nStatistik Deskriptif:")
display(df.describe())

In [ ]:
# Visualisasi distribusi variabel target
plt.figure(figsize=(6, 4))
sns.countplot(x='target', data=df)
plt.title('Distribusi Variabel Target (0: Sehat, 1: Sakit Jantung)')
plt.xlabel('Status Penyakit Jantung')
plt.ylabel('Jumlah Pasien')
plt.show()

# Visualisasi distribusi beberapa fitur numerik penting
features_to_plot = ['age', 'trestbps', 'chol', 'thalach']
df[features_to_plot].hist(figsize=(10, 8), bins=20)
plt.suptitle('Distribusi Fitur Numerik Penting')
plt.show()

# Visualisasi korelasi antar fitur menggunakan heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Heatmap Korelasi Antar Fitur')
plt.show()

## **4. Data Preparation**
Tahap ini meliputi pemisahan data menjadi set pelatihan dan pengujian, serta standarisasi fitur numerik.

In [ ]:
# Memisahkan fitur (X) dan target (y)
X = df.drop('target', axis=1)
y = df['target']

# Membagi data menjadi data latih (80%) dan data uji (20%)
# Menggunakan stratify=y untuk menjaga proporsi kelas target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Ukuran data latih: {X_train.shape}")
print(f"Ukuran data uji: {X_test.shape}")

# Standarisasi fitur numerik
numerical_features = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']
scaler = StandardScaler()

# Melakukan fit dan transform pada data latih
X_train[numerical_features] = scaler.fit_transform(X_train[numerical_features])

# Hanya melakukan transform pada data uji (menggunakan scaler yang sudah di-fit)
X_test[numerical_features] = scaler.transform(X_test[numerical_features])

print("\nData setelah standarisasi (5 baris pertama dari data latih):")
display(X_train.head())

## **5. Modeling**
Kita akan membangun, melatih, dan mengevaluasi dua model klasifikasi: Logistic Regression dan Random Forest.

In [ ]:
# Inisialisasi model
log_reg = LogisticRegression(random_state=42)
rand_forest = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)

# Melatih model Logistic Regression
log_reg.fit(X_train, y_train)

# Melatih model Random Forest
rand_forest.fit(X_train, y_train)

## **6. Evaluation**
Evaluasi performa kedua model menggunakan berbagai metrik pada data uji.

In [ ]:
# Fungsi untuk mengevaluasi model dan menampilkan hasilnya
def evaluate_model(model, X_test, y_test, model_name):
    y_pred = model.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    print(f"--- Evaluasi Model: {model_name} ---")
    print(f"Akurasi: {accuracy:.2f}")
    print(f"Presisi: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1-Score: {f1:.2f}\n")
    
    # Menampilkan classification report
    print("Laporan Klasifikasi:")
    print(classification_report(y_test, y_pred))
    
    # Menampilkan confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(6, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Sehat', 'Sakit Jantung'],
                yticklabels=['Sehat', 'Sakit Jantung'])
    plt.title(f'Confusion Matrix - {model_name}')
    plt.xlabel('Prediksi')
    plt.ylabel('Aktual')
    plt.show()
    
    return {'Akurasi': accuracy, 'Presisi': precision, 'Recall': recall, 'F1-Score': f1}

# Mengevaluasi kedua model
log_reg_metrics = evaluate_model(log_reg, X_test, y_test, "Logistic Regression")
rand_forest_metrics = evaluate_model(rand_forest, X_test, y_test, "Random Forest")

### **Perbandingan Hasil Evaluasi Model**

In [ ]:
# Membuat DataFrame untuk membandingkan metrik
metrics_df = pd.DataFrame({
    'Logistic Regression': log_reg_metrics,
    'Random Forest': rand_forest_metrics
})

print("Perbandingan Metrik Evaluasi:")
display(metrics_df.round(2))